In [1]:
from corsikaio import CorsikaParticleFile
from tqdm import tqdm
from pathlib import Path
import os
import numpy as np

In [5]:
path_to_res = "/dicos_ui_home/antonpr/work/corsika_runs/11_simple_run/results"
path_to_res = Path(path_to_res)

resut_files = []
for file in path_to_res.glob("DAT*[!.long]"):
    resut_files.append(file)

In [12]:
with CorsikaParticleFile(resut_files[0]) as f:
        
        counter = 0
        for e in f:
                      
            particle_data = e.particles

            particles_in_subblock = particle_data['particle_description']
            pids_in_subblock = (particles_in_subblock - (particles_in_subblock % 1000)) / 1000
            
            for pdg in get_pdg:
                
                show_corsika_id = corsika_id_dict[pdg]
                show_corsika_id_bar = corsika_id_dict[-pdg]
                
                part_mass = mass_dict_GeV[np.abs(pdg)]

                filter_pid_inds = np.where((pids_in_subblock == show_corsika_id))[0]

                filtered_obslevels = np.array(particles_in_subblock % 10)[filter_pid_inds] 
                
                filtered_pz = particle_data['pz'][filter_pid_inds]
                filtered_px = particle_data['px'][filter_pid_inds]
                filtered_py = particle_data['py'][filter_pid_inds]
                
                filtered_x = particle_data['x'][filter_pid_inds]
                filtered_y = particle_data['y'][filter_pid_inds]
                
                filtered_unique_ids = particles_in_subblock[filter_pid_inds]
                
                
                r = np.sqrt(filtered_x**2 + filtered_y**2)
                

                filtered_pT = np.sqrt(filtered_px**2 + filtered_py**2)
                
                filtered_ptot = np.sqrt(filtered_px**2 + filtered_py**2 + filtered_pz**2)
            

                filtered_theta = np.arctan2(filtered_pT, filtered_pz)
                filtered_phi = np.arctan2(filtered_py, filtered_px)
                
                init_dir = np.array([np.ones_like(filtered_phi) * np.sin(np.radians(theta)), 
                           np.zeros_like(filtered_phi) * np.sin(np.radians(theta)),
                           np.ones_like(filtered_phi) * np.cos(np.radians(theta))]).T
                
                final_dir = np.array([np.cos(filtered_phi) * np.sin(filtered_theta),
                                     np.sin(filtered_phi) * np.sin(filtered_theta),
                                     np.cos(filtered_theta)]).T
                   
                dot_products = np.arccos(np.sum(init_dir * final_dir, axis=1))
                
                filtered_energies = np.sqrt(filtered_pT**2 + filtered_pz**2 + part_mass**2) - part_mass

                for obslev_id in level_range:

                    this_obslev_inds = np.where(filtered_obslevels == obslev_id)[0]

                    all_thetas_corsika[pdg][obslev_id].extend(dot_products[this_obslev_inds])

                    all_energies_corsika[pdg][obslev_id].extend(filtered_energies[this_obslev_inds])
                    
                    all_unique_pids[pdg][obslev_id].extend(filtered_unique_ids[this_obslev_inds])

[75052.  5521. 68521. 68521. 67521. 69521. 75052.  5521. 68521. 69531.
 66531. 68531. 68541. 69521. 66521. 69521. 66521. 68521. 67521. 69521.]
[75.  5. 68. 68. 67. 69. 75.  5. 68. 69. 66. 68. 68. 69. 66. 69. 66. 68.
 67. 69.]
[68521. 68531. 67531. 68541. 69541. 68541. 69551. 68541. 68031. 75003.
  5031. 69531. 66531. 68531. 69541. 69541. 66541. 68541.]
[68. 68. 67. 68. 69. 68. 69. 68. 68. 75.  5. 69. 66. 68. 69. 69. 66. 68.]
[69531. 66531. 66541. 69541. 68541. 67541. 76052.  6521. 69521. 69521.
 66521. 68521.]
[69. 66. 66. 69. 68. 67. 76.  6. 69. 69. 66. 68.]
[76053.  6531. 69531. 75053.  5531. 68531. 66531. 68531. 67531. 76053.
  6531. 69531. 75053.  5531. 68531. 68531. 69531. 67541. 68541. 69551.
 76055.  6551. 69551. 67551. 69551. 69541. 66541. 68541.]
[76.  6. 69. 75.  5. 68. 66. 68. 67. 76.  6. 69. 75.  5. 68. 68. 69. 67.
 68. 69. 76.  6. 69. 67. 69. 69. 66. 68.]
[68021. 69021. 69521. 69021. 68021. 67021. 68521. 67521. 69521. 75052.
  5521. 68521. 68521. 67521.]
[68. 69. 69. 69. 6

In [9]:
particles_in_subblock

array([68521., 69521., 69531., 67541., 76054.,  6541., 69531., 68531.,
       69541., 66541., 68541., 69571., 66571., 68571., 76054.,  6541.,
       69541., 75054.,  5541., 68541.], dtype=float32)